In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

#from trading_system_copy import TradingSystem
#from trading_optimization_copy import OptimizeTradingSystem

from trading_system import TradingSystem as TradingSystem



In [12]:
hub1_name = "ttf"
hub2_name = "the"
validation_size = 250
test_size = 250
window_size = 5
#model = "vecm"
model = "ann"
#model = "ann_long"
#model = "tvecm_long_t1"

In [13]:
cointegration_params = {
    "hub1_name": hub1_name,
    "hub2_name": hub2_name,
    "model": "ols",
    "validation_size": validation_size,
    "test_size": test_size,
    "window_size": window_size,
    "mode": "validation",
    "strategy": "cointegration",
    "cointegration_rolling_window": 250
}

params = {
    "hub1_name": hub1_name,
    "hub2_name": hub2_name,
    "model": model,
    "validation_size": validation_size,
    "test_size": test_size,
    "window_size": window_size,
    "mode": "validation",
}

In [14]:
validation_ts = TradingSystem(**params)

In [15]:
#volatility = "rolling"
volatility = "sGARCH_sstd"
#volatility = "fiGARCH_sstd"
#volatility = "rolling"

cointegration_study_params = {
    "volatility": volatility,
    "rolling_window_range": [5,60,5],
    "lower_threshold_range": [0,2.0,0.01],
    "criteria": "mean_returns",
    "min_trades": 10,
    "verbose": True,
    "plot": True
}

study_params = {
    "volatility": volatility,
    "rolling_window_range": [5,60,5],
    "lower_threshold_range": [0.0,2.0,0.01],
    "criteria": "mean_returns",
    "min_trades": 10,
    "verbose": True,
    "plot": True
}
best_profit, best_rolling_window, best_lower_threshold = validation_ts.study(**study_params)

Best lower threshold: 1.31
Profit: 14.05

Mean returns: 0.11%
Standard deviation of returns: 0.56%
Sharpe ratio: 0.20
Mean returns with transaction costs: 0.11%
Confidence interval (returns): 0.04% - 0.18%
Confidence interval (returns with transaction costs): 0.04% - 0.18%

Win rate for returns: 33.20%
No trade rate for returns: 42.40%
Loss rate for returns: 24.40%

Win rate for returns with transaction costs: 33.20%
No trade rate for returns with transaction costs: 42.40%
Loss rate for returns with transaction costs: 24.40%


In [16]:
params["mode"] = "test"
test_ts = TradingSystem(**params)
test_ts.run_trading_system(volatility=volatility,
                      rolling_window=best_rolling_window, 
                      lower_threshold=best_lower_threshold,
                      naive=False, 
                      verbose=True, 
                      plot=True)

Profit: 4.78

Mean returns: 0.08%
Standard deviation of returns: 0.41%
Sharpe ratio: 0.18
Mean returns with transaction costs: 0.08%
Confidence interval (returns): 0.02% - 0.13%
Confidence interval (returns with transaction costs): 0.02% - 0.13%

Win rate for returns: 13.20%
No trade rate for returns: 84.00%
Loss rate for returns: 2.80%

Win rate for returns with transaction costs: 13.20%
No trade rate for returns with transaction costs: 84.00%
Loss rate for returns with transaction costs: 2.80%


In [160]:
ann_returns = test_ts.returns

In [161]:
ann_hub1_predictions = test_ts.hub1_predictions
ann_hub2_predictions = test_ts.hub2_predictions

hub1_actuals = test_ts.hub1_actuals
hub2_actuals = test_ts.hub2_actuals

In [162]:
params["mode"] = "validation"
params["model"] = "vecm"
validation_ts = TradingSystem(**params)
best_profit, best_rolling_window, best_lower_threshold = validation_ts.study(**study_params)


Best lower threshold: 0.04
Profit: 260.02

Mean returns: 1.04%
Standard deviation of returns: 4.94%
Sharpe ratio: 0.21
Mean returns with transaction costs: 1.04%
Confidence interval (returns): 0.43% - 1.65%
Confidence interval (returns with transaction costs): 0.43% - 1.65%

Win rate for returns: 54.80%
No trade rate for returns: 9.60%
Loss rate for returns: 35.60%

Win rate for returns with transaction costs: 54.80%
No trade rate for returns with transaction costs: 9.60%
Loss rate for returns with transaction costs: 35.60%


In [163]:
params["mode"] = "test"
test_ts = TradingSystem(**params)
test_ts.run_trading_system(volatility=volatility,
                      rolling_window=best_rolling_window, 
                      lower_threshold=best_lower_threshold,
                      naive=False, 
                      verbose=True, 
                      plot=True)

Profit: 18.87

Mean returns: 0.29%
Standard deviation of returns: 1.52%
Sharpe ratio: 0.19
Mean returns with transaction costs: 0.29%
Confidence interval (returns): 0.11% - 0.48%
Confidence interval (returns with transaction costs): 0.11% - 0.48%

Win rate for returns: 56.80%
No trade rate for returns: 10.80%
Loss rate for returns: 32.40%

Win rate for returns with transaction costs: 56.80%
No trade rate for returns with transaction costs: 10.80%
Loss rate for returns with transaction costs: 32.40%


In [164]:
vecm_returns = test_ts.returns

In [165]:
vecm_hub1_predictions = test_ts.hub1_predictions
vecm_hub2_predictions = test_ts.hub2_predictions

In [166]:
fig = go.Figure()

# Add traces for each series
fig.add_trace(go.Scatter(x=vecm_hub1_predictions.index, y=vecm_hub1_predictions[hub1_name], mode='lines', name='VECM Hub1 Predictions'))
fig.add_trace(go.Scatter(x=ann_hub1_predictions.index, y=ann_hub1_predictions[hub1_name], mode='lines', name='ANN Hub1 Predictions'))
fig.add_trace(go.Scatter(x=hub1_actuals.index, y=hub1_actuals[hub1_name], mode='lines', name='Hub1 Actuals'))

# Update layout
fig.update_layout(title='Hub1 Predictions and Actuals',
                  xaxis_title='Date',
                  yaxis_title='Value',
                  legend_title='Series')

# Show plot
fig.show()

In [167]:
fig = go.Figure()

# Add traces for each series
fig.add_trace(go.Scatter(x=vecm_hub2_predictions.index, y=vecm_hub2_predictions[hub2_name], mode='lines', name='VECM Hub2 Predictions'))
fig.add_trace(go.Scatter(x=ann_hub2_predictions.index, y=ann_hub2_predictions[hub2_name], mode='lines', name='ANN Hub2 Predictions'))
fig.add_trace(go.Scatter(x=hub2_actuals.index, y=hub2_actuals[hub2_name], mode='lines', name='Hub2 Actuals'))

# Update layout
fig.update_layout(title='Hub2 Predictions and Actuals',
                  xaxis_title='Date',
                  yaxis_title='Value',
                  legend_title='Series')

# Show plot
fig.show()

In [168]:

predicted_spread_ann = ann_hub1_predictions[hub1_name] - ann_hub2_predictions[hub2_name]
predicted_spread_vecm = vecm_hub1_predictions[hub1_name] - vecm_hub2_predictions[hub2_name]
actual_spread = hub1_actuals[hub1_name] - hub2_actuals[hub2_name]

fig = go.Figure()


fig.add_trace(go.Scatter(x=predicted_spread_ann.index, y=predicted_spread_ann, mode='lines', name='ANN Predicted Spread'))
fig.add_trace(go.Scatter(x=predicted_spread_vecm.index, y=predicted_spread_vecm, mode='lines', name='VECM Predicted Spread'))
fig.add_trace(go.Scatter(x=actual_spread.index, y=actual_spread, mode='lines', name='Actual Spread'))

fig.update_layout(title='Predicted and Actual Spread (Hub1 - Hub2)',
                  xaxis_title='Date',
                  yaxis_title='Spread',
                  legend_title='Series')


fig.show()

In [170]:

# Calculate the 5-lag of actuals
hub1_actuals_lagged = hub1_actuals.shift(5)

# Calculate the predicted returns for ANN
predicted_returns_hub1_ann = ann_hub1_predictions[hub1_name] / hub1_actuals_lagged[hub1_name]

# Calculate the predicted returns for VECM
predicted_returns_hub1_vecm = vecm_hub1_predictions[hub1_name] / hub1_actuals_lagged[hub1_name]

actual_returns = hub1_actuals[hub1_name] / hub1_actuals_lagged[hub1_name]

# Drop NaN values resulting from the lag
predicted_returns_hub1_ann = predicted_returns_hub1_ann.dropna()
predicted_returns_hub1_vecm = predicted_returns_hub1_vecm.dropna()

# Plot the predicted returns
fig = go.Figure()

fig.add_trace(go.Scatter(x=predicted_returns_hub1_ann.index, y=predicted_returns_hub1_ann, mode='lines', name='ANN Hub1 Predicted Returns'))
fig.add_trace(go.Scatter(x=predicted_returns_hub1_vecm.index, y=predicted_returns_hub1_vecm, mode='lines', name='VECM Hub1 Predicted Returns'))
fig.add_trace(go.Scatter(x=actual_returns.index, y=actual_returns, mode='lines', name='Hub1 Actual Returns'))

fig.update_layout(title='Hub1 Predicted Returns',
                xaxis_title='Date',
                yaxis_title='Predicted Returns',
                legend_title='Series')

fig.show()

In [171]:
# Calculate the 5-lag of actuals for hub2
hub2_actuals_lagged = hub2_actuals.shift(5)

# Calculate the predicted returns for ANN
predicted_returns_hub2_ann = ann_hub2_predictions[hub2_name] / hub2_actuals_lagged[hub2_name]

# Calculate the predicted returns for VECM
predicted_returns_hub2_vecm = vecm_hub2_predictions[hub2_name] / hub2_actuals_lagged[hub2_name]

actual_returns_hub2 = hub2_actuals[hub2_name] / hub2_actuals_lagged[hub2_name]

# Drop NaN values resulting from the lag
predicted_returns_hub2_ann = predicted_returns_hub2_ann.dropna()
predicted_returns_hub2_vecm = predicted_returns_hub2_vecm.dropna()

# Plot the predicted returns for hub2
fig = go.Figure()

fig.add_trace(go.Scatter(x=predicted_returns_hub2_ann.index, y=predicted_returns_hub2_ann, mode='lines', name='ANN Hub2 Predicted Returns'))
fig.add_trace(go.Scatter(x=predicted_returns_hub2_vecm.index, y=predicted_returns_hub2_vecm, mode='lines', name='VECM Hub2 Predicted Returns'))
fig.add_trace(go.Scatter(x=actual_returns_hub2.index, y=actual_returns_hub2, mode='lines', name='Hub2 Actual Returns'))

fig.update_layout(title='Hub2 Predicted Returns',
                xaxis_title='Date',
                yaxis_title='Predicted Returns',
                legend_title='Series')

fig.show()

In [172]:
# Calculate the spread for ANN and VECM predicted returns
predicted_spread_ann_returns = predicted_returns_hub1_ann - predicted_returns_hub2_ann
predicted_spread_vecm_returns = predicted_returns_hub1_vecm - predicted_returns_hub2_vecm
actual_return_spread = actual_returns - actual_returns_hub2

# Create a plotly figure
fig = go.Figure()

# Add traces for each series
fig.add_trace(go.Scatter(x=predicted_spread_ann_returns.index, y=predicted_spread_ann_returns, mode='lines', name='ANN Predicted Return Spread'))
fig.add_trace(go.Scatter(x=predicted_spread_vecm_returns.index, y=predicted_spread_vecm_returns, mode='lines', name='VECM Predicted Return Spread'))
fig.add_trace(go.Scatter(x=actual_return_spread.index, y=actual_return_spread, mode='lines', name='Actual Return Spread'))

# Update layout
fig.update_layout(title='Predicted and Actual Return Spread',
                  xaxis_title='Date',
                  yaxis_title='Spread',
                  legend_title='Series')

# Show plot
fig.show()

In [119]:
np.mean(ann_returns), np.mean(vecm_returns)

(0.00318223126318801, 0.002294544365868678)

In [80]:
vecm_returns = [0 if vecm < 0 and ann == 0 else vecm for ann, vecm in zip(ann_returns, vecm_returns)]

# Calculate the mean of vecm_returns
mean_vecm_returns = np.mean(vecm_returns)

print(f"Mean of vecm_returns: {mean_vecm_returns}")

Mean of vecm_returns: 0.0036239176692995077


In [81]:
returns_df = pd.DataFrame({
    'ann_returns': ann_returns,
    'vecm_returns': vecm_returns
})
returns_df

,ann_returns,vecm_returns
0,-0.013789,-0.013789
1,-0.016445,-0.016445
2,-0.014037,-0.014037
3,-0.016963,-0.016963
4,0.000385,0.000385
5,0.004863,0.004863
6,0.002661,0.002661
7,0.003305,0.003305
8,0.014224,0.014224
9,0.010602,0.010602


In [82]:
filtered_return_df = returns_df[returns_df['vecm_returns'] > -0.04]

In [83]:
filtered_return_df.describe()

,ann_returns,vecm_returns
count,247.000000,247.000000
mean,0.004004,0.004306
std,0.013872,0.013269
min,-0.036017,-0.037529
25%,-0.003133,0.000000
50%,0.002532,0.002503
75%,0.010115,0.009754
max,0.056008,0.056008


In [46]:
# Multiply each return by 100
returns_df['ann_returns'] = returns_df['ann_returns'] * 100
returns_df['vecm_returns'] = returns_df['vecm_returns'] * 100

# Create a scatter plot
fig = px.scatter(returns_df, x='ann_returns', y='vecm_returns', title='Scatter plot of ANN vs VECM returns')
fig.show()

In [50]:
print(len(ann_returns), len(vecm_returns))

250 250


In [53]:
# Count records where both returns are positive
both_positive = sum(1 for ann, vecm in zip(ann_returns, vecm_returns) if ann > 0 and vecm > 0)

# Count records where both returns are negative
both_negative = sum(1 for ann, vecm in zip(ann_returns, vecm_returns) if ann < 0 and vecm < 0)

# Count records where both returns are zero
both_zero = sum(1 for ann, vecm in zip(ann_returns, vecm_returns) if ann == 0 and vecm == 0)

# Count records where vecm returns are positive and ann returns are negative or zero
vecm_positive_ann_negative = sum(1 for ann, vecm in zip(ann_returns, vecm_returns) if ann < 0 and vecm > 0)
vecm_positive_ann_zero = sum(1 for ann, vecm in zip(ann_returns, vecm_returns) if ann == 0 and vecm > 0)

# Count records where vecm returns are negative and ann returns are positive or zero
vecm_negative_ann_positive = sum(1 for ann, vecm in zip(ann_returns, vecm_returns) if ann > 0 and vecm < 0)
vecm_negative_ann_zero = sum(1 for ann, vecm in zip(ann_returns, vecm_returns) if ann == 0 and vecm < 0)

# Count records where vecm returns are zero and ann returns are positive or negative
vecm_zero_ann_positive = sum(1 for ann, vecm in zip(ann_returns, vecm_returns) if ann > 0 and vecm == 0)
vecm_zero_ann_negative = sum(1 for ann, vecm in zip(ann_returns, vecm_returns) if ann < 0 and vecm == 0)

# Print results
print(f"Both positive: {both_positive}")
print(f"Vecm positive and Ann negative: {vecm_positive_ann_negative}")
print(f"Vecm positive and Ann zero: {vecm_positive_ann_zero}")
print(f"Vecm negative and Ann positive: {vecm_negative_ann_positive}")
print(f"Vecm negative and Ann zero: {vecm_negative_ann_zero}")
print(f"Vecm zero and Ann positive: {vecm_zero_ann_positive}")
print(f"Vecm zero and Ann negative: {vecm_zero_ann_negative}")
print(f"Both negative: {both_negative}")
print(f"Both zero: {both_zero}")

# Verify the total count
total_count = (
    both_positive +
    vecm_positive_ann_negative +
    vecm_positive_ann_zero +
    vecm_negative_ann_positive +
    vecm_negative_ann_zero +
    vecm_zero_ann_positive +
    vecm_zero_ann_negative +
    both_negative +
    both_zero
)

print(f"Total count: {total_count}, Expected: {len(ann_returns)}")


Both positive: 108
Vecm positive and Ann negative: 29
Vecm positive and Ann zero: 5
Vecm negative and Ann positive: 25
Vecm negative and Ann zero: 18
Vecm zero and Ann positive: 12
Vecm zero and Ann negative: 10
Both negative: 38
Both zero: 5
Total count: 250, Expected: 250


In [55]:
# Set entries of vecm_returns to 0 if vecm is negative and ann is zero
vecm_returns = [0 if vecm < 0 and ann == 0 else vecm for ann, vecm in zip(ann_returns, vecm_returns)]

# Calculate the mean of vecm_returns
mean_vecm_returns = np.mean(vecm_returns)

print(f"Mean of vecm_returns: {mean_vecm_returns}")

Mean of vecm_returns: 0.0036239176692995077


In [40]:
# Calculate the number of equal elements
equal_elements = sum(1 for ann, vecm in zip(ann_returns, vecm_returns) if ann == vecm)

# Calculate the percentage
percentage_equal = (equal_elements / len(ann_returns)) * 100

print(f"Percentage of equal elements: {percentage_equal:.2f}%")

Percentage of equal elements: 60.40%


In [42]:
# Calculate the percentage of positive returns for ann_returns
positive_ann_returns = sum(1 for ret in ann_returns if ret > 0)
percentage_positive_ann = (positive_ann_returns / len(ann_returns)) * 100

# Calculate the percentage of positive returns for vecm_returns
positive_vecm_returns = sum(1 for ret in vecm_returns if ret > 0)
percentage_positive_vecm = (positive_vecm_returns / len(vecm_returns)) * 100


print(f"Percentage of positive ann_returns: {percentage_positive_ann:.2f}%")
print(f"Percentage of positive vecm_returns: {percentage_positive_vecm:.2f}%")

Percentage of positive ann_returns: 58.00%
Percentage of positive vecm_returns: 56.80%


In [43]:
# Calculate the percentage of negative returns for ann_returns
negative_ann_returns = sum(1 for ret in ann_returns if ret < 0)
percentage_negative_ann = (negative_ann_returns / len(ann_returns)) * 100

# Calculate the percentage of negative returns for vecm_returns
negative_vecm_returns = sum(1 for ret in vecm_returns if ret < 0)
percentage_negative_vecm = (negative_vecm_returns / len(vecm_returns)) * 100

print(f"Percentage of negative ann_returns: {percentage_negative_ann:.2f}%")
print(f"Percentage of negative vecm_returns: {percentage_negative_vecm:.2f}%")

Percentage of negative ann_returns: 30.80%
Percentage of negative vecm_returns: 32.40%


In [7]:
print("Window_size", window_size)
print("Rolling_window", best_rolling_window)
print("Threshold", best_lower_threshold)

mean = f"{test_ts.get_returns_stats()[0]:.2f}"
print("Mean: ", mean)

std = f"{test_ts.get_returns_stats()[1]:.2f}"
print("Standard Deviation: ", std)

CI = test_ts.get_returns_stats()[3]
CI_f = f"{CI[0]:.2f}% -{CI[1]:.2f}%"
print("Confidence Interval: ", CI_f)

trade_rate = f"{(1- test_ts.get_trade_rates()['no_trade_rate_returns'])*100:.1f}"
print("Trade rate: ", trade_rate)

data = {
    "Pair": f"{hub1_name} - {hub2_name}",
    "Window_size": window_size,
    "Rolling_window": best_rolling_window,
    "Threshold": best_lower_threshold,
    "Mean": mean,
    "Standard_Deviation": std,
    "Confidence_Interval": CI_f,
    "Trade_rate": trade_rate
}

# Convert the dictionary into a pandas dataframe
df = pd.DataFrame([data])



Window_size 5
Rolling_window 55
Threshold 0.1
Mean:  0.24
Standard Deviation:  1.28
Confidence Interval:  0.08% -0.39%
Trade rate:  84.8


In [8]:
df

,Pair,Window_size,Rolling_window,Threshold,Mean,Standard_Deviation,Confidence_Interval,Trade_rate
0,ttf - nbp,5,55,0.1,0.24,1.28,0.08% -0.39%,84.8


In [93]:
cointegration_params = {
    "validation_size": validation_size,
    "test_size": test_size,
    "strategy": "cointegration",
    "cointegration_rolling_window": 250
}

forecasting_params = {
    "strategy": "forecasting",
    "validation_size": validation_size,
    "test_size": test_size,
}

cointegration_study_params = {
    "rolling_window_range": [5,60,5],
    "lower_threshold_range": [0.0,2.0,0.01],
}

forecasting_study_params = {
    "rolling_window_range": [5,60,5],
    "lower_threshold_range": [0.0,2.0,0.01],
}

validation_size = 250
test_size = 250
window_size = 5
model = "ann_long"
models = ["ann_long", "ann", "vecm", "tvecm_long_t1", "ols"]
pairs = ["ttf-the", "ttf-nbp", "the-nbp"]
window_sizes = [5]
volatility_models = ["rolling", "sGARCH_norm","sGARCH_sstd", "fiGARCH_sstd", "gjrGARCH_sstd"]

df = pd.DataFrame()

for model in models:
    for window_size in window_sizes:
        for pair in pairs:
            for vol_model in volatility_models:

                hub1_name = pair.split("-")[0]
                hub2_name = pair.split("-")[1]

                params = cointegration_params if model == "ols" else forecasting_params
                study_params = cointegration_study_params if model == "ols" else forecasting_study_params

                
                params["hub1_name"] = hub1_name
                params["hub2_name"] = hub2_name
                params["model"] = model
                params["window_size"] = window_size

                study_params["volatility"] = vol_model
                study_params["criteria"] = "mean_returns"
                study_params["min_trades"] = 10
                study_params["verbose"] = False
                study_params["plot"] = False

                params["mode"] = "validation"
                validation_ts = TradingSystem(**params)
                best_profit, best_rolling_window, best_lower_threshold = validation_ts.study(**study_params)

                params["mode"] = "test"
                test_ts = TradingSystem(**params)
                test_ts.run_trading_system(volatility=vol_model,
                                    rolling_window=best_rolling_window, 
                                    lower_threshold=best_lower_threshold, 
                                    naive=False, 
                                    verbose=False, 
                                    plot=False)


                mean = f"{test_ts.get_returns_stats()[0]:.3f}"
                std = f"{test_ts.get_returns_stats()[1]:.3f}"

                CI = test_ts.get_returns_stats()[3]
                CI_f = f"{CI[0]:.3f}% -{CI[1]:.3f}%"

                trade_rate = f"{(1- test_ts.get_trade_rates()['no_trade_rate_returns'])*100:.1f}"

                data = {
                    "Model": model,
                    "Pair": f"{hub1_name} - {hub2_name}",
                    "Volatility_model": vol_model,
                    "Window_size": window_size,
                    "Rolling_window": best_rolling_window,
                    "Threshold": best_lower_threshold,
                    "Mean": mean,
                    "Standard_Deviation": std,
                    "Confidence_Interval": CI_f,
                    "Trade_rate": trade_rate
                }

                # Convert the dictionary into a pandas dataframe
                df = pd.concat([df, pd.DataFrame([data])])


In [94]:
df

,Model,Pair,Volatility_model,Window_size,Rolling_window,Threshold,Mean,Standard_Deviation,Confidence_Interval,Trade_rate
0,ann_long,ttf - the,rolling,5,40,0.72,0.115,0.583,0.042% -0.187%,67.2
0,ann_long,ttf - the,sGARCH_norm,5,None,0.02,0.108,0.631,0.030% -0.186%,98.4
0,ann_long,ttf - the,sGARCH_sstd,5,None,0.02,0.108,0.631,0.030% -0.186%,98.4
0,ann_long,ttf - the,fiGARCH_sstd,5,None,0.02,0.108,0.631,0.030% -0.186%,98.4
0,ann_long,ttf - the,gjrGARCH_sstd,5,None,0.02,0.108,0.631,0.030% -0.186%,98.4
0,ann_long,ttf - nbp,rolling,5,55,1.00,0.012,0.974,-0.109% -0.133%,39.2
0,ann_long,ttf - nbp,sGARCH_norm,5,None,1.59,0.028,0.672,-0.055% -0.111%,13.2
0,ann_long,ttf - nbp,sGARCH_sstd,5,None,1.19,0.059,0.800,-0.040% -0.158%,21.2
0,ann_long,ttf - nbp,fiGARCH_sstd,5,None,1.34,0.047,0.755,-0.046% -0.141%,19.2
0,ann_long,ttf - nbp,gjrGARCH_sstd,5,None,1.19,0.055,0.803,-0.044% -0.155%,21.6


In [95]:
sorted_df = df.sort_values(by=["Pair", "Mean"], ascending=False)

In [96]:
pd.set_option('display.max_rows', None)
sorted_df

,Model,Pair,Volatility_model,Window_size,Rolling_window,Threshold,Mean,Standard_Deviation,Confidence_Interval,Trade_rate
0,tvecm_long_t1,ttf - the,sGARCH_norm,5,None,0.07,0.178,0.595,0.104% -0.251%,86.8
0,tvecm_long_t1,ttf - the,sGARCH_sstd,5,None,0.06,0.178,0.595,0.104% -0.251%,86.8
0,tvecm_long_t1,ttf - the,fiGARCH_sstd,5,None,0.06,0.178,0.595,0.104% -0.251%,86.8
0,tvecm_long_t1,ttf - the,gjrGARCH_sstd,5,None,0.06,0.178,0.595,0.104% -0.251%,86.8
0,tvecm_long_t1,ttf - the,rolling,5,5,0.15,0.172,0.599,0.098% -0.246%,89.2
0,vecm,ttf - the,fiGARCH_sstd,5,None,0.11,0.149,0.597,0.075% -0.223%,84.0
0,vecm,ttf - the,sGARCH_norm,5,None,0.11,0.148,0.597,0.074% -0.222%,84.4
0,vecm,ttf - the,sGARCH_sstd,5,None,0.11,0.148,0.597,0.074% -0.222%,84.4
0,vecm,ttf - the,gjrGARCH_sstd,5,None,0.11,0.148,0.597,0.074% -0.222%,84.4
0,vecm,ttf - the,rolling,5,40,0.17,0.136,0.590,0.063% -0.209%,78.8
